## Volbrain deeplesion verzamelen

This code is for organizing the Volbrain.net output of the DeepLesion Algorythm. It runs over the zip files in a specific folder, and extracts necessary information from the enclosed CSVs and puts themm all together in a table

In [ ]:
#load necessary libraries
import pandas as pd
import numpy as np 
import zipfile
import os
from zipfile import ZipFile
import glob

In [ ]:
#navigate to zipfolders and intended folder for output
path_volbrain = 'Z:/VolBrain'
output_folder = "Z:/VolBrain/Separate_CSV_Deeplesion"


In [ ]:
#create a list to store the CSV headers in
all_data = []

In [ ]:
dataframes = []
from io import BytesIO


for zip_file in glob.glob(os.path.join(path_volbrain, '*.zip')):
    # split by underscores
    participant_id = zip_file.split("_")
    # concatenate BRICK with 00X
    participant_id = participant_id[1] + "_" + participant_id[2]
    with ZipFile(zip_file, "r") as zf:
        #get the list of files inside the zip (assumes 1 csv per zip folder)
        for entry_name in zf.namelist():
            if entry_name.endswith(".csv"):
                entry = zf.read(entry_name)
                df = pd.read_csv(BytesIO(entry), sep=';')
                df['Participant Id'] = participant_id
                dataframes.append(df)

In [ ]:
concat_df = pd.concat(dataframes)
concat_df

In [ ]:
#move the participant id row to the front. Now it's last. This is important for upload in castor.
concat_df = concat_df.loc[:, [concat_df.columns[-1]] + list(concat_df.columns[:-1])]


In [ ]:
#Now make the column names castor appropriate. Do not change the "Participant Id" column of course. dl stands for DeepLesion algorithm
concat_df.columns = [
    col if col == "Participant Id" else "dl_" + col.replace(" ", "_") + "_T0"
    for col in concat_df.columns
]

#For castor, male is 1 and female is 2. Replace these values
concat_df["dl_Sex_T0"] = concat_df["dl_Sex_T0"].replace({"Female": 2, "Male": 1})


In [ ]:
concat_df.to_csv('Z:/castor_proof_files/volbrains_castor.csv')